In [1]:
from src.data_preparation.data_preparation import read_eeg_file
from scipy import signal
from scipy import linalg
from scipy import stats
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pyriemann.utils.mean as rie_mean
import numpy as np

/home/mauricio/development/environments/anaconda3/envs/motor-imagery/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Define some parameters

In [2]:
FS = 100
TIME_LENGTH = int(FS * 2.5)
TIME_WINDOW = int(FS * 2.5)
DATA_FOLDER = "data/si-bci/bci-iii-dataset-iv-a"
CSP_COMPONENTS = 8
METRIC_COVARIANCE_ESTIMATION = "logeuclid"

Define a fifth Butterworth bandpass filter

In [3]:
sos = signal.butter(5, [8, 30], analog=False, btype="band", output="sos", fs=FS)

Define the function to compute the spatial filters

In [4]:
def compute_spatial_filters(left_data, right_data):
    n_channels = left_data.shape[2]
    cov_shape = (n_channels, n_channels)

    # Estimate the covariance matrix of every trial
    n_left_trials = left_data.shape[0]
    cov = np.zeros((n_left_trials, *cov_shape))
    for n_trial in range(n_left_trials):
        trial = signal.sosfilt(sos, left_data[n_trial], axis=0)
        cov[n_trial] = np.cov(np.transpose(trial))

    # calculate average of covariance matrix
    cov_1 = rie_mean.mean_covariance(cov, metric=METRIC_COVARIANCE_ESTIMATION)

    # Estimate the covariance matrix of every trial
    n_right_trials = right_data.shape[0]
    cov = np.zeros((n_right_trials, *cov_shape))
    for n_trial in range(n_right_trials):
        trial = signal.sosfilt(sos, right_data[n_trial], axis=0)
        cov[n_trial] = np.cov(np.transpose(trial))

    # calculate average of covariance matrix
    cov_2 = rie_mean.mean_covariance(cov, metric=METRIC_COVARIANCE_ESTIMATION)

    # Solve the generalized eigenvalue problem
    n_pairs = CSP_COMPONENTS // 2
    w, vr = linalg.eig(cov_1, cov_2, right=True)
    w = np.abs(w)
    sorted_indexes = np.argsort(w)
    chosen_indexes = np.zeros(2 * n_pairs).astype(int)
    chosen_indexes[0:n_pairs] = sorted_indexes[0:n_pairs]
    chosen_indexes[n_pairs:2 * n_pairs] = sorted_indexes[-n_pairs:]

    return vr[:, chosen_indexes]


Define the function to extract the log variance and katz fractal dimension features

In [5]:
def extract_features(left_data, right_data):
    W = compute_spatial_filters(left_data, right_data)

    X = np.concatenate((left_data, right_data))
    n_trials = X.shape[0]
    features = {
        "CSP": np.zeros((n_trials, CSP_COMPONENTS)),
        "KATZ_FRACTAL": np.zeros((n_trials, CSP_COMPONENTS))
    }

    for n_trial in range(n_trials):
        x = X[n_trial]
        x = signal.sosfilt(sos, x, axis=0)
        z = np.dot(np.transpose(W), np.transpose(x))
        features["CSP"][n_trial] = np.log(np.divide(np.var(z, axis=1), np.sum(np.var(z, axis=1))))
        for n_component in range(CSP_COMPONENTS):
            features["KATZ_FRACTAL"][n_trial, n_component] = katz_fractal(z[n_component])

    return features

Define the function to compute the katz fractal dimension feature

In [6]:
def katz_fractal(x):
    n = len(x) - 1

    # Calculate the total length L of the signal
    L = 0
    for n_i in range(n):
        # Use the Euclidean distance formula to obtain the distance between the consecutive points
        x_distance = 1
        y_distance = (x[n_i] - x[n_i + 1]) ** 2
        L = L + np.sqrt(x_distance + y_distance)

    # Calculate the diameter of the signal, that is the farthest distance between the starting point to any other point
    d = np.zeros(n)
    for n_i in range(n):
        # Use the Euclidean distance formula to obtain the distance between the points to the origin
        x_distance = (n_i + 1) ** 2
        y_distance = (x[0] - x[n_i + 1]) ** 2
        d[n_i] = np.sqrt(x_distance + y_distance)
    d = np.max(d)

    ln = np.log10(n)
    return ln / (np.log10(d / L) + ln)


Define the functions to make the predictions

In [7]:
def lda_prediction_score(X_train, Y_train, X_test, Y_test):
    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train, Y_train)
    predictions = lda.predict(X_test)
    return accuracy_score(Y_test, predictions)

def svm_prediction_score(X_train, Y_train, X_test, Y_test):
    svm = SVC(C=.8, gamma="scale", kernel="rbf")
    svm.fit(X_train, Y_train)
    predictions = svm.predict(X_test)
    return accuracy_score(Y_test, predictions)

Generate the spatial filters separately for each subject,
and extract the features.

The algorithm uses 4 subjects for train, and 1 subject for test.

In [8]:
subjects = range(1, 6)
subjects_set = set(subjects)
accuracies = {
    "SVM": {
        "CSP": np.zeros(len(subjects)),
        "KATZ_FRACTAL": np.zeros(len(subjects))
    },
    "LDA": {
        "CSP": np.zeros(len(subjects)),
        "KATZ_FRACTAL": np.zeros(len(subjects))
    }
}

for test_subject in subjects:
    print("Subject: ", test_subject)
    train_subjects = list(subjects_set - {test_subject})

    # Load training data
    path_train_files = [(f"{DATA_FOLDER}/left-hand-subject-{train_subject}.csv",
                        f"{DATA_FOLDER}/right-hand-subject-{train_subject}.csv")
                        for train_subject in train_subjects]
    train_data_by_subject = [read_eeg_file(left_data_file, right_data_file, TIME_LENGTH, TIME_WINDOW)
                             for left_data_file, right_data_file in path_train_files]

    # Load test data
    left_data_file = f"{DATA_FOLDER}/left-hand-subject-{test_subject}.csv"
    right_data_file = f"{DATA_FOLDER}/right-hand-subject-{test_subject}.csv"
    test_data = read_eeg_file(left_data_file, right_data_file, TIME_LENGTH, TIME_WINDOW, training=False)

    train_data = {
        "Y": [],
        "CSP": None,
        "KATZ_FRACTAL": None
    }
    for (i, train_data_subject) in enumerate(train_data_by_subject):
        train_data_subject.left_data = signal.sosfilt(sos, train_data_subject.left_data, axis=1)
        train_data_subject.right_data = signal.sosfilt(sos, train_data_subject.right_data, axis=1)

        train_data["Y"] = np.concatenate((train_data["Y"], train_data_subject.labels))
        train_features = extract_features(train_data_subject.left_data, train_data_subject.right_data)
        if i == 0:
            train_data["CSP"] = train_features["CSP"]
            train_data["KATZ_FRACTAL"] = train_features["KATZ_FRACTAL"]
        else:
            train_data["CSP"] = np.concatenate((train_data["CSP"], train_features["CSP"]))
            train_data["KATZ_FRACTAL"] = np.concatenate((train_data["KATZ_FRACTAL"], train_features["KATZ_FRACTAL"]))

    test_data.F = extract_features(test_data.left_data, test_data.right_data)

    # Normalize the features
    train_data["CSP"] = stats.zscore(train_data["CSP"], axis=0)
    train_data["KATZ_FRACTAL"] = stats.zscore(train_data["KATZ_FRACTAL"], axis=0)
    test_data.F["CSP"] = stats.zscore(test_data.F["CSP"], axis=0)
    test_data.F["KATZ_FRACTAL"] = stats.zscore(test_data.F["KATZ_FRACTAL"], axis=0)

    accuracy_index = test_subject - 1
    csp_parameters = (train_data["CSP"], train_data["Y"], test_data.F["CSP"], test_data.labels)
    katz_fractal_parameters = (train_data["KATZ_FRACTAL"], train_data["Y"], 
                               test_data.F["KATZ_FRACTAL"], test_data.labels)
    accuracies["SVM"]["CSP"][accuracy_index] = svm_prediction_score(*csp_parameters)
    accuracies["SVM"]["KATZ_FRACTAL"][accuracy_index] = svm_prediction_score(*katz_fractal_parameters)
    accuracies["LDA"]["CSP"][accuracy_index] = lda_prediction_score(*csp_parameters)
    accuracies["LDA"]["KATZ_FRACTAL"][accuracy_index] = lda_prediction_score(*katz_fractal_parameters)

Subject:  1
Subject:  2
Subject:  3
Subject:  4
Subject:  5


Print the accuracies obtained

In [9]:
for classifier_algorithm in accuracies.keys():
    print("=========================")
    print(f"{classifier_algorithm}\n")
    for feature_extraction_method in accuracies[classifier_algorithm].keys():
        print(feature_extraction_method)
        for subject, cv_accuracies in enumerate(accuracies[classifier_algorithm][feature_extraction_method]):
            acc_mean = np.mean(cv_accuracies) * 100
            print(f"\tSubject {subject + 1} average accuracy: {acc_mean:.4f}")
        average_acc_mean = np.mean(accuracies[classifier_algorithm][feature_extraction_method]) * 100
        average_acc_std = np.std(accuracies[classifier_algorithm][feature_extraction_method]) * 100
        print(f"\tAverage accuracy: {average_acc_mean:.4f} +/- {average_acc_std:.4f}")
        print()

SVM

CSP
	Subject 1 average accuracy: 90.3571
	Subject 2 average accuracy: 95.3571
	Subject 3 average accuracy: 66.0714
	Subject 4 average accuracy: 95.0000
	Subject 5 average accuracy: 93.9286
	Average accuracy: 88.1429 +/- 11.1767

KATZ_FRACTAL
	Subject 1 average accuracy: 90.3571
	Subject 2 average accuracy: 96.0714
	Subject 3 average accuracy: 73.9286
	Subject 4 average accuracy: 95.0000
	Subject 5 average accuracy: 95.7143
	Average accuracy: 90.2143 +/- 8.3989

LDA

CSP
	Subject 1 average accuracy: 90.0000
	Subject 2 average accuracy: 92.5000
	Subject 3 average accuracy: 66.4286
	Subject 4 average accuracy: 93.9286
	Subject 5 average accuracy: 95.0000
	Average accuracy: 87.5714 +/- 10.7033

KATZ_FRACTAL
	Subject 1 average accuracy: 90.7143
	Subject 2 average accuracy: 95.0000
	Subject 3 average accuracy: 76.0714
	Subject 4 average accuracy: 95.3571
	Subject 5 average accuracy: 97.1429
	Average accuracy: 90.8571 +/- 7.6884

